# 문제점 해결

### 얼굴의 회전/기울임에 따른 문제

얼굴이 좌우, 위아래로 회전/기울어질 경우 스티커의 모양은 그대로이므로 부자연스럽다.  
얼굴의 회전/기울임에 따라 스티커의 모양도 변형해주면 자연스럽게 보일 것으로 보인다.   
이를 위한 방법으로는 dlib에서 제공하는 얼굴의 포인트중 좌상, 좌하, 우상, 우하단의 점을 뽑아 사각형으로 행렬을 얻어낸다.   
이 행렬을 기준으로 스티커이미지를 AFFINE 변환하면 얼굴형 사각형에 따라 스티커도 동일하게 변화하므로 자연스럽게 보일 수 있다.   

### 얼굴 크기에 따른 문제

LMS에 제공된 소스 중 얼굴 영역 사각형의 넓이가 100 이하인 경우 추론하지 않도록 하고 있다.   
이 제한을 제외하면 일부 해소 된다.   
다만, dlib자체가 가진 분해 능력은 여전히 제한점으로 남게 된다.

### 사람 수에 따른 문제

dlib가 제공하는 detection갯수를 조정한다.   

# 수정된 코드

LMS에 제공된 파일 중 newaddsticker.py만 수정되었다.   
좌우 영역을 넘어갔을때 발생되는 오류의 수정은 이미 되어 있으므로 칼만 필터 적용만 수정되었다.

In [ ]:
import dlib
import cv2

import numpy as np          # 추가
from kpkf import tracker    # 추가

box_tracker = tracker(num_points=2, sys_err=0.5, measure_err=1000) # 추가

def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1)
    if len(dlib_rects) < 1:
        return img_orig

    # 추가
    if len(dlib_rects) == 1:
        bbox = dlib_rects[0]
        list_input = [(bbox.left(), bbox.top()), (bbox.right(), bbox.bottom())]
        np_estimate = np.array(box_tracker.process(list_input))
        np_est_points = np_estimate.reshape(2, 3)[:,:2].astype(int)
        l,t,r,b = np_est_points.flatten()
        # if (b-t)*(r-l) > 100: 제외
        dlib_rects[0] = dlib.rectangle(left=l,top=t,right=r,bottom=b)
    
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
    
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[30][0]
        y = landmark[30][1] - dlib_rect.width()//2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break
    
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    
    refined_x = x - w // 2
    refined_y = y - h
    
    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]

    img_bgr = img_orig.copy()
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr

# 결론

칼만 필터가 적용되었을 때 스티커의 흔들림은 줄어들었다. 하지만 약간의 흔들림은 지속 되었다.   
또한, 프레임 중간에 얼굴 영역이 검출되지 않는 프레임이 있는 경우 스티커가 깜빡이는 현상을 없앨 수 없었다.   